In [2]:
from func.objects import get_spark_object
from func.io_functions import read_data,write_data
from func.profile import data_profiling
from func.common_functions import get_properties,plot_profile_report,warning_image,warning_message_with_icon
import re

In [3]:
from pyspark.sql.functions import col
import pandas as pd
import re


In [4]:
%%time
spark = get_spark_object('test2')
spark

CPU times: total: 0 ns
Wall time: 15 ms


In [5]:
ip_props = {'ip_type': 'file',
 'file_type': 'csv',
 'ip_file_path': r"C:\Users\ptpl-338\Desktop\mp_8\test_data\sample.csv",
 'header': 'True',
 'delimiter': ','}

In [6]:
df = read_data(spark,ip_props)
try:
    df = auto_id(df,spark)
    df_columns = df.columns
    # Use the select() method to apply the upper() function to all columns
    df = df.select(*[col(column).alias(column.upper()) for column in df_columns])
except:
    pass


In [7]:
df.limit(5).toPandas()

,Address,DOB,Name,Telephone No,Email id,Pincode,Sex,Age,pan
0,"DURVAKUR, PLOT NO. 4, 4TH CROSS, SHASTRI NAGA...",13-05-1901,KESHRI CHAND DAGA,9705754774,sravanthisankoji@gmail.com,590001,male,22,AAABP0970
1,"DURVAKUR, PLOT NO. 4, 4TH CROSS, SHASTRI NAGA...",13-05-1902,"KMR.PALAK-LA,SHRI VINUKANT D.SHAH-LP",9908732790,nikithakolipaka@gmail.com,590001,female,55,AACB4868Q
2,"SAKTHI 234, KILPAUK GARDEN ROAD CHENNAI 600010",13-05-1903,"KMR.PRANAV(LA),SHRI RAMESH O. PARIKH(LP)",7659965779,srijaulli@gmail.com,600010,male,66,AAACE4354L
3,"AKSHAYA, NEAR GUJARAT BHAVAN BAILAPPANAVAR NAG...",13-05-1904,"KR. JAYESH H. BHATT(LA),SHRI HARIT-",9948412522,ravalikabongoni@gmail.com,0,female,60,AAACM3730C
4,"ANASWARA|PO: PALLOOR|MAHE, PUDUCHERRY - UT|673...",13-05-1905,SH SHRI BHAGWAN,9618844701,arunakula@gmail.com,None,male,22,AAACN7k86N


In [8]:
df.printSchema()

root
 |-- Address: string (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Telephone No: string (nullable = true)
 |-- Email id: string (nullable = true)
 |-- Pincode: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- pan: string (nullable = true)



In [77]:
res_df = pd.DataFrame(df.dtypes, columns = ['Column Name','Data type'])

In [78]:
res_df

,Column Name,Data type
0,Address,string
1,DOB,string
2,Name,string
3,Telephone No,string
4,Email id,string
5,Pincode,string
6,Sex,string
7,Age,string
8,pan,string


In [79]:
missing_values = {} 
unique_values = {} 
for index, column in enumerate(df.columns):
    if column in df.columns:    # check string columns with None and Null values
        missing_count = df.filter(col(column).eqNullSafe(None) | col(column).isNull()).count()
        unique_count = df.select(col(column)).distinct().count()
        missing_values.update({column:missing_count})
        unique_values.update({column:unique_count})


In [80]:
total_count = int(df.count())

res_df['Missing_Values'] = list(missing_values.values())
res_df['Unique_Values'] = list(unique_values.values())
res_df['Missing_Values_%'] = res_df['Missing_Values'].apply(lambda x: round(int(x)/total_count * 100,2) )
res_df['Unique_Values_%'] = res_df['Unique_Values'].apply(lambda x: round(int(x)/total_count * 100,2) )

In [81]:
res_df

,Column Name,Data type,Missing_Values,Unique_Values,Missing_Values_%,Unique_Values_%
0,Address,string,105,98443,0.10,89.15
1,DOB,string,2256,2877,2.04,2.61
2,Name,string,10135,90613,9.18,82.06
3,Telephone No,string,93150,5298,84.36,4.80
4,Email id,string,101479,525,91.90,0.48
5,Pincode,string,7526,9928,6.82,8.99
6,Sex,string,109830,71,99.46,0.06
7,Age,string,109844,554,99.48,0.50
8,pan,string,80429,3046,72.84,2.76


In [115]:
def identify_string_type(input_str):
    if  input_str is None or input_str == '' or input_str =='NA' or input_str =='N/A':
        return 'N/A'
    elif re.match(r'^\d+(\.\d+)?$', input_str):  # Matches numbers, including decimals
        return 'Number'
    elif re.match(r'^\d{4}-\d{2}-\d{2}$', input_str):  # Matches DOB in yyyy-mm-dd format
        return 'DOB'
    elif re.match(r'^\d{2}-\d{2}-\d{4}$', input_str):  # Matches DOB in yyyy-mm-dd format
        return 'DOB'
    else:
        return 'String'

# Test cases
strings = ["123", "45.67", "2023-08-06", "invalid", "1995-01-25","01-05-1995"]
for s in strings:
    print(f"'{s}' is identified as: {identify_string_type(s)}")


'123' is identified as: Number
'45.67' is identified as: Number
'2023-08-06' is identified as: DOB
'invalid' is identified as: String
'1995-01-25' is identified as: DOB
'01-05-1995' is identified as: DOB


In [116]:
pand_df = df.limit(5).toPandas()

In [117]:
pand_df

,Address,DOB,Name,Telephone No,Email id,Pincode,Sex,Age,pan
0,"DURVAKUR, PLOT NO. 4, 4TH CROSS, SHASTRI NAGA...",13-05-1901,KESHRI CHAND DAGA,9705754774.00,sravanthisankoji@gmail.com,590001.00,male,22.00,AAABP0970
1,"DURVAKUR, PLOT NO. 4, 4TH CROSS, SHASTRI NAGA...",13-05-1902,"KMR.PALAK-LA,SHRI VINUKANT D.SHAH-LP",9908732790.00,nikithakolipaka@gmail.com,590001.00,female,55.00,AACB4868Q
2,"SAKTHI 234, KILPAUK GARDEN ROAD CHENNAI 600010",13-05-1903,"KMR.PRANAV(LA),SHRI RAMESH O. PARIKH(LP)",7659965779.00,srijaulli@gmail.com,600010.00,male,66.00,AAACE4354L
3,"AKSHAYA, NEAR GUJARAT BHAVAN BAILAPPANAVAR NAG...",13-05-1904,"KR. JAYESH H. BHATT(LA),SHRI HARIT-",9948412522.00,ravalikabongoni@gmail.com,0.00,female,60.00,AAACM3730C
4,"ANASWARA|PO: PALLOOR|MAHE, PUDUCHERRY - UT|673...",13-05-1905,SH SHRI BHAGWAN,9618844701.00,arunakula@gmail.com,None,male,22.00,AAACN7k86N


In [118]:
for col in pand_df.columns:
    print(col)
    pand_df[col] = pand_df[col].apply(lambda x: identify_string_type(x))

Address
DOB
Name
Telephone No
Email id
Pincode
Sex
Age
pan


In [119]:
pand_df

,Address,DOB,Name,Telephone No,Email id,Pincode,Sex,Age,pan
0,String,DOB,String,Number,String,Number,String,Number,String
1,String,DOB,String,Number,String,Number,String,Number,String
2,String,DOB,String,Number,String,Number,String,Number,String
3,String,DOB,String,Number,String,Number,String,Number,String
4,String,DOB,String,Number,String,N/A,String,Number,String


In [1]:
pip list

Package                   Version
------------------------- -----------
altair                    5.0.1
altgraph                  0.17.3
anyio                     3.7.0
argon2-cffi               21.3.0
argon2-cffi-bindings      21.2.0
arrow                     1.2.3
asttokens                 2.2.1
attrs                     23.1.0
backcall                  0.2.0
backports.zoneinfo        0.2.1
beautifulsoup4            4.12.2
bleach                    6.0.0
blinker                   1.6.2
cachetools                5.3.1
certifi                   2023.5.7
cffi                      1.15.1
charset-normalizer        3.1.0
click                     8.1.3
colorama                  0.4.6
comm                      0.1.3
configparser              5.3.0
contourpy                 1.1.0
cycler                    0.11.0
debugpy                   1.6.7
decorator                 5.1.1
defusedxml                0.7.1
et-xmlfile                1.1.0
exceptiongroup            1.1.1
executing             


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
